In [1]:
#Roto League Analysis 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import io


In [2]:
# !pip install pulp
import pulp

https://www.fangraphs.com/auctiontool.aspx?type=pit&proj=atc&pos=1,1,1,1,4,1,0,0,0,2,6,4,0,0,0&dollars=260&teams=12&mp=20&msp=5&mrp=5&mb=1&split=70&points=c|0,1,2,4,7,16|2,3,4,8,13,14&lg=MLB&rep=0&drp=20&pp=C,SS,2B,3B,OF,1B&players=19427|17,15474|9,13543|17,15998|19,19709|4,13125|32,16556|19,15172|6,14212|12,9874|5,19556|4,12533|4,11739|24,13145|6,10324|24,13613|5,15640|19,20099|4,19612|4,18345|3,20043|3,16252|31,19755|10,19374|12,12564|26,18401|19,18882|5,10954|36,17479|9

In [68]:
# #import data from local files
# from google.colab import files
# uploaded = files.upload()

In [3]:
#transform imports to dataframes
# df_hitter = pd.read_csv(io.BytesIO(uploaded['fg_hitter.csv']))
df_hitter_in = pd.read_csv('fg_hitter.csv')
df_hitter_in.rename(columns={df_hitter_in.columns[0]: "PlayerName" },inplace=True)

df_pitcher_in = pd.read_csv('fg_pitcher.csv')
df_pitcher_in.rename(columns={df_pitcher_in.columns[0]: "PlayerName" },inplace=True)

In [4]:
#Player Floor
df_hitter = df_hitter_in.loc[(df_hitter_in['ADP']<350).copy()]

df_pitcher = df_pitcher_in.loc[df_pitcher_in['ADP']<350].copy()


In [5]:
#PrimaryPosition
df_hitter['PP'] = df_hitter['POS']
positionOrder = ['C','SS','2B','3B','OF','1B','SP','DH']
df_hitter = df_hitter.copy()
for pos in positionOrder:
    df_hitter.loc[df_hitter['PP'].str.contains(pos), 'PP'] = pos
    
df_pitcher['PP'] = df_pitcher['POS']
positionOrder2 = ['SP','RP','DH']
df_pitcher = df_pitcher.copy()
for pos in positionOrder2:
    df_pitcher.loc[df_pitcher['PP'].str.contains(pos), 'PP'] = pos

# eventually need some logic for multi-position eligibility

C:\Users\coste\AppData\Local\Temp/ipykernel_9260/3319194379.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hitter['PP'] = df_hitter['POS']


In [6]:
df_hitter.head()

,PlayerName,Team,POS,ADP,PA,mAVG,mRBI,mR,mSB,mHR,PTS,aPOS,Dollars,PlayerId,PP
0,Fernando Tatis Jr.,SDP,SS/OF,1.84,614.480,4.681894,6.777359,8.107917,7.591735,8.358853,35.517758,11.702951,48.220709,19709,SS
1,Juan Soto,WSN,OF,4.37,662.744,9.301478,6.809459,8.386328,0.992570,5.011574,30.501410,12.357495,43.858905,20123,OF
2,Vladimir Guerrero Jr.,TOR,1B/DH,5.87,650.002,8.808487,7.495808,6.823711,-1.931637,7.270142,28.466511,11.547262,41.013773,19611,1B
3,Trea Turner,LAD,2B/SS,1.54,650.502,8.703384,1.936769,7.405949,9.690717,0.244229,27.981047,11.581955,40.563002,16252,SS
4,Jose Ramirez,CLE,3B,3.63,633.006,1.718156,5.345840,5.231959,6.991717,4.512873,23.800544,12.544743,37.345287,13510,3B


In [7]:
#get hitter
def getHitter(playerName):
    return (df_hitter.loc[df_hitter['PlayerName']==playerName])

In [8]:
#get pitcher
def getPitcher(playerName):
    return (df_hitter.loc[df_hitter['PlayerName']==playerName])

In [10]:
#add hitter to team
def addHitter(playerNames,team):
    for i in range(len(playerNames)):
        name=playerNames[i]
        team=team.append(getHitter(name))
    return team

In [11]:
#add pitcher to team
def addPitcher(playerNames,team):
    for i in range(len(playerNames)):
        name=playerNames[i]
        team=team.append(getPitcher(name))
    return team

In [12]:
#remove player from team
def dropPlayer(playerName,team):
    target=team.loc[team['PlayerName']==playerName]
    team=team.drop(target)
    return team

In [13]:
#add cost to player
def addPlayerCost(playerName,cost):
    df_hitter.loc[df_hitter['PlayerName'] == playerName, 'Cost'] = cost


In [14]:
#add cost to pitcher
def addPitcherCost(playerName,cost):
    df_pitcher.loc[df_pitcher['PlayerName'] == playerName, 'Cost'] = cost

In [15]:
#create roster
myTeam=pd.DataFrame()
myRotation=pd.DataFrame()

In [16]:
if 'Cost' not in df_hitter:
    df_hitter['Cost']=None

if 'Cost' not in df_pitcher:
    df_pitcher['Cost']=None
    
availHitter=df_hitter[df_hitter['Cost'].isnull()]
availPitcher=df_pitcher[df_pitcher['Cost'].isnull()]

In [82]:
# def hittingOptimizer(df):
#     # initialize variables
#     playerPositions = df['POS'].tolist()
#     playerNames = df['PlayerName'].tolist()
#     playerCosts = tuple(df['Dollars']) #using own projection prices
#     playerPoints = tuple(df['PTS'])
    
#     # valueDict = {}
#     # for category in df_hitter.columns:
#     #     # if string category starts with 'm'
#     #     if category[0] == 'm':
#     #         # add category to valueDict
#     #         valueDict[category] = tuple(df[category])
   
#     mAVG = tuple(df['mAVG'])
#     mRBI = tuple(df['mRBI'])
#     mR = tuple(df['mR'])
#     mHR = tuple(df['mHR'])
#     mSB = tuple(df['mSB'])
    
#     salaryCap = 260*.7-10
#     numOfPlayers = range(len(playerCosts))
    
#     # formulate linear programming problem
#     problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)
#     # create variable to represent each player
#     playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")

#     # create objective: maximize sum of player points
#     # problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)

#     # more complex objective function
#     #  	mAVG 	mRBI 	mR 	mHR 	mOPS 	mSBCS
#     # problem += pulp.lpSum(playerInLineup[i] * 
#     #                       (scipy.stats.cauchy.cdf(mAVG[i]) + 
#     #                        scipy.stats.cauchy.cdf(mRBI[i]) +
#     #                        scipy.stats.cauchy.cdf(mR[i]) +
#     #                        scipy.stats.cauchy.cdf(mHR[i]) +
#     #                        scipy.stats.cauchy.cdf(mSB[i])
#     #                       ) 
#     #                       for i in numOfPlayers)
    
#     problem += pulp.lpSum(playerInLineup[i] * 
#                           (scipy.stats.cauchy.cdf(mAVG[i]) + 
#                            scipy.stats.cauchy.cdf(mRBI[i]) +
#                            scipy.stats.cauchy.cdf(mR[i]) +
#                            scipy.stats.cauchy.cdf(mHR[i]) +
#                            scipy.stats.cauchy.cdf(mSB[i])
#                           ) 
#                           for i in numOfPlayers)
                                                
    
#     # constraint: each player can only be chosen at most once
#     for i in numOfPlayers:
#         problem += pulp.lpSum(playerInLineup[i]) <= 1
#     # constraints: sum of player costs must be less than or equal to the salary cap
#     problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap
    
#     # constraint: teams must have 6 players drafted (4 flyers + catcher)
#     #problem += pulp.lpSum(problem.variables()) == 
#     problem += pulp.lpSum(playerInLineup[i] for i in numOfPlayers) == 6 # num of players in lineup
    
#     #add keeper players from my team    
#     keep = tuple(df['Cost'] > 0)
#     problem += pulp.lpSum(playerInLineup[i] * keep[i] for i in numOfPlayers) == sum(keep)

#     # positions
#     # constraint: 1 = C
#     catchers = tuple(df['PP'].str.contains('C'))
#     problem += pulp.lpSum(playerInLineup[i] * catchers[i] for i in numOfPlayers) >= 0
    
#     # constraint: 1 <= number of 1b
#     fb = tuple(df['PP'].str.contains('1B'))
#     problem += pulp.lpSum(playerInLineup[i] * fb[i] for i in numOfPlayers) >= 0

#     # constraint: 1 <= number of 2b
#     sb = tuple(df['PP'].str.contains('2B'))
#     problem += pulp.lpSum(sb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

#     # constraint: 1 <= number of 3b
#     tb = tuple(df['PP'].str.contains('3B'))
#     problem += pulp.lpSum(tb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

#     # constraint: 1 <= number of SS
#     ss = tuple(df['PP'].str.contains('SS'))
#     problem += pulp.lpSum(playerInLineup[i] * ss[i] for i in numOfPlayers) >= 1

#     # constraint: 4 <= number of OF
#     of = tuple(df['PP'].str.contains('OF'))
#     problem += pulp.lpSum(playerInLineup[i] * of[i] for i in numOfPlayers) >= 2
    
#     dh = tuple(df['PP']==('DH'))
#     problem += pulp.lpSum(playerInLineup[i] * dh[i] for i in numOfPlayers) == 0  
     
#     # output the calculated predicted lineup
#     cost = 0
#     points = 0

#     predicted = list()
#     if problem.solve() == 1:
#         print ("Current Optimal Lineup")
#         for pos in range(len(numOfPlayers)):
#             if playerInLineup[pos].value() == 1:
#                 name = playerNames[pos]
#                 predicted.append(name)
#                 cost += playerCosts[pos]
#                 points += playerPoints[pos]
               
#                 print ('%25s, Position = %2s, Price = %5.f, Points = %3.2f' \
#                       % (playerNames[pos], playerPositions[pos], playerCosts[pos],
#                          playerPoints[pos]))
#         print ('\nTotal Team Cost: %5d\nTotal Team Points: %5.2f' % (cost, points))
        
#     else:
#         print ('Error finding solution')
        
#     return(predicted,cost,points)

In [117]:
# df = df_hitter.copy()
# stat_columns = [col for col in df.columns if col[0] == 'm']
# player_stats = df.loc[:, stat_columns]
# player_stats
# target_player_stats = player_stats.iloc[[0,2,3]]
# # target_player_stats
# # sum player stat columns
# player_stats_summed = target_player_stats.sum(axis=0)
# # print(player_stats_summed)
# # apply cauchy distribution to player_summed_columns
# player_stats_summed_cauchy = scipy.stats.cauchy.cdf(player_stats_summed)
# # print(player_stats_summed_cauchy)
# # # sum total cauchy distribution
# player_stats_summed_cauchy_sum = player_stats_summed_cauchy.sum()
# print(player_stats_summed_cauchy_sum)


4.9110818204447595


In [17]:
def hittingOptimizer(df):
    # initialize variables
    playerPositions = df['POS'].tolist()
    playerNames = df['PlayerName'].tolist()
    playerCosts = tuple(df['Dollars']) #using own projection prices
    playerPoints = tuple(df['PTS'])
    
    # valueDict = {}
    # for category in df_hitter.columns:
    #     # if string category starts with 'm'
    #     if category[0] == 'm':
    #         # add category to valueDict
    #         valueDict[category] = tuple(df[category])
    
    stat_columns = [col for col in df.columns if col[0] == 'm']
    player_stats = df.loc[:, stat_columns]

    salaryCap = 260*.7-10
    numOfPlayers = range(len(playerCosts))
    
    # formulate linear programming problem
    problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)
    # create variable to represent each player
    playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")

    # create objective: maximize sum of player points
    # problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)

    # more complex objective function
    #  	mAVG 	mRBI 	mR 	mHR 	mOPS 	mSBCS
    
    # Sum player stats and take cauchy distribution of total
    problem += pulp.lpSum((scipy.stats.cauchy.cdf((playerInLineup[i] * player_stats).sum(axis=0)).sum()) for i in numOfPlayers)
                                                
    
    # constraint: each player can only be chosen at most once
    for i in numOfPlayers:
        problem += pulp.lpSum(playerInLineup[i]) <= 1
    # constraints: sum of player costs must be less than or equal to the salary cap
    problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap
    
    # constraint: teams must have 6 players drafted (4 flyers + catcher)
    #problem += pulp.lpSum(problem.variables()) == 
    problem += pulp.lpSum(playerInLineup[i] for i in numOfPlayers) == 6 # num of players in lineup
    
    #add keeper players from my team    
    keep = tuple(df['Cost'] > 0)
    problem += pulp.lpSum(playerInLineup[i] * keep[i] for i in numOfPlayers) == sum(keep)

    # positions
    # constraint: 1 = C
    catchers = tuple(df['PP'].str.contains('C'))
    problem += pulp.lpSum(playerInLineup[i] * catchers[i] for i in numOfPlayers) >= 0
    
    # constraint: 1 <= number of 1b
    fb = tuple(df['PP'].str.contains('1B'))
    problem += pulp.lpSum(playerInLineup[i] * fb[i] for i in numOfPlayers) >= 0

    # constraint: 1 <= number of 2b
    sb = tuple(df['PP'].str.contains('2B'))
    problem += pulp.lpSum(sb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of 3b
    tb = tuple(df['PP'].str.contains('3B'))
    problem += pulp.lpSum(tb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of SS
    ss = tuple(df['PP'].str.contains('SS'))
    problem += pulp.lpSum(playerInLineup[i] * ss[i] for i in numOfPlayers) >= 1

    # constraint: 4 <= number of OF
    of = tuple(df['PP'].str.contains('OF'))
    problem += pulp.lpSum(playerInLineup[i] * of[i] for i in numOfPlayers) >= 2
    
    dh = tuple(df['PP']==('DH'))
    problem += pulp.lpSum(playerInLineup[i] * dh[i] for i in numOfPlayers) == 0  
     
    # output the calculated predicted lineup
    cost = 0
    points = 0

    predicted = list()
    if problem.solve() == 1:
        print ("Current Optimal Lineup")
        for pos in range(len(numOfPlayers)):
            if playerInLineup[pos].value() == 1:
                name = playerNames[pos]
                predicted.append(name)
                cost += playerCosts[pos]
                points += playerPoints[pos]
               
                print ('%25s, Position = %2s, Price = %5.f, Points = %3.2f' \
                      % (playerNames[pos], playerPositions[pos], playerCosts[pos],
                         playerPoints[pos]))
        print ('\nTotal Team Cost: %5d\nTotal Team Points: %5.2f' % (cost, points))
        
    else:
        print ('Error finding solution')
        
    return(predicted,cost,points)

In [18]:
def playerOptimizer(df_hitter, df_pitcher):
    # initialize variables
    playerPositions = df_hitter['PP'].tolist()
    # playerIDs = df_hitter['PlayerID'].tolist()
    playerNames = df_hitter['PlayerName'].tolist()
    playerCosts = tuple(df_hitter['Dollars']) #using own projection prices
    playerPoints = tuple(df_hitter['PTS'])
    pitcherCosts = tuple(df_pitcher['Dollars']) #using own projection prices
    pitcherPoints = tuple(df_pitcher['PTS'])
    pitcherNames = df_pitcher['PlayerName'].tolist()
    # pitcherIDs = df_pitcher['PlayerID'].tolist()
    
    # valueDict = {}
    # for category in df_hitter.columns:
    #     # if string category starts with 'm'
    #     if category[0] == 'm':
    #         # add category to valueDict
    #         valueDict[category] = tuple(df[category])
   
    mAVG = tuple(df_hitter['mAVG'])
    mRBI = tuple(df_hitter['mRBI'])
    mR = tuple(df_hitter['mR'])
    mHR = tuple(df_hitter['mHR'])
    mSB = tuple(df_hitter['mSB'])

    mW = tuple(df_pitcher['mW'])
    mK = tuple(df_pitcher['mSO'])
    mERA = tuple(df_pitcher['mERA'])
    mWHIP = tuple(df_pitcher['mWHIP'])
    mSV = tuple(df_pitcher['mSV'])
    
    salaryCap = 260-10
    numOfPlayers = range(len(playerCosts))
    numOfPitchers = range(len(pitcherCosts))
    
    # formulate linear programming problem
    problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)
    # create variable to represent each player
    playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")
    pitcherInLineup = pulp.LpVariable.dicts("Pitchers", [i for i in numOfPitchers], 0, 1, cat="Binary")

    # create objective: maximize sum of player points
    # problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)

    # more complex objective function using cauchy distribution
    problem += pulp.lpSum([(i,j) for i in numOfPlayers for j in numOfPitchers 
    (playerInLineup[i] * (scipy.stats.cauchy.cdf(mAVG[i]) + scipy.stats.cauchy.cdf(mRBI[i]) + 
    scipy.stats.cauchy.cdf(mR[i]) + scipy.stats.cauchy.cdf(mHR[i]) + scipy.stats.cauchy.cdf(mSB[i]))) +   
    (pitcherInLineup[j] * (scipy.stats.cauchy.cdf(mW[j]) + scipy.stats.cauchy.cdf(mK[j])+ scipy.stats.cauchy.cdf(mERA[j])+ scipy.stats.cauchy.cdf(mWHIP[j])+ scipy.stats.cauchy.cdf(mSV[j])))])
    #  (playerInLineup[i] * (scipy.stats.cauchy.cdf(mAVG[i]) + scipy.stats.cauchy.cdf(mRBI[i]) + scipy.stats.cauchy.cdf(mR[i]) + scipy.stats.cauchy.cdf(mHR[i]) + scipy.stats.cauchy.cdf(mSB[i]))) 
    # + (pitcherInLineup[j] * (scipy.stats.cauchy.cdf(mW[j]) + scipy.stats.cauchy.cdf(mK[j])+ scipy.stats.cauchy.cdf(mERA[j])+ scipy.stats.cauchy.cdf(mWHIP[j])+ scipy.stats.cauchy.cdf(mSV[j]))) for (i, j) in numOfPlayers for j in numOfPitchers])                      
                          
    # constraint: each player can only be chosen at most once
    for i in numOfPlayers:
        problem += pulp.lpSum(playerInLineup[i]) <= 1
    # constraints: sum of player costs must be less than or equal to the salary cap
    problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap
    
    # constraint: teams must have 6 players drafted (4 flyers + catcher)
    #problem += pulp.lpSum(problem.variables()) == 
    problem += pulp.lpSum(playerInLineup[i] for i in numOfPlayers) == 6 # num of players in lineup
    
    #add keeper players from my team    
    keep = tuple(df_hitter['Cost'] > 0)
    problem += pulp.lpSum(playerInLineup[i] * keep[i] for i in numOfPlayers) == sum(keep)

    # positions
    # constraint: 1 = C
    catchers = tuple(df_hitter['PP'].str.contains('C'))
    problem += pulp.lpSum(playerInLineup[i] * catchers[i] for i in numOfPlayers) >= 0
    
    # constraint: 1 <= number of 1b
    fb = tuple(df_hitter['PP'].str.contains('1B'))
    problem += pulp.lpSum(playerInLineup[i] * fb[i] for i in numOfPlayers) >= 0

    # constraint: 1 <= number of 2b
    sb = tuple(df_hitter['PP'].str.contains('2B'))
    problem += pulp.lpSum(sb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of 3b
    tb = tuple(df_hitter['PP'].str.contains('3B'))
    problem += pulp.lpSum(tb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of SS
    ss = tuple(df_hitter['PP'].str.contains('SS'))
    problem += pulp.lpSum(playerInLineup[i] * ss[i] for i in numOfPlayers) >= 1

    # constraint: 4 <= number of OF
    of = tuple(df_hitter['PP'].str.contains('OF'))
    problem += pulp.lpSum(playerInLineup[i] * of[i] for i in numOfPlayers) >= 2
    
    dh = tuple(df_hitter['PP']==('DH'))
    problem += pulp.lpSum(playerInLineup[i] * dh[i] for i in numOfPlayers) >= 0

    # constraint: 6 <= number of pitchers
    pit = tuple(df_pitcher)
    problem += pulp.lpSum(pitcherInLineup[i] * pit[i] for i in numOfPitchers) >= 6

     
    # output the calculated predicted lineup
    cost = 0
    points = 0

    predicted = list()
    if problem.solve() == 1:
        print ("Current Optimal Lineup")
        for pos in range(len(numOfPlayers)):
            if playerInLineup[pos].value() == 1:
                name = playerNames[pos]
                predicted.append(name)
                cost += playerCosts[pos]
                points += playerPoints[pos]
               
                print ('%25s, Position = %2s, Price = %5.f, Points = %3.2f' \
                      % (playerNames[pos], playerPositions[pos], playerCosts[pos],
                         playerPoints[pos]))
        print ('\nTotal Hitter Cost: %5d\nTotal Hitter Points: %5.2f' % (cost, points))

        for pos in range(len(numOfPitchers)):
            if pitcherInLineup[pos].value() ==1:
                name = pitcherNames[pos]
                predicted.append(name)
                cost += pitcherCosts[pos]
                points += pitcherPoints[pos]

            print ('%25s, Price = %5.f, Points = %3.2f' \
                      % (pitcherNames[pos], pitcherCosts[pos],
                         pitcherPoints[pos]))
        print ('\nTotal Pitcher Cost: %5d\nTotal Pitcher Points: %5.2f' % (cost, points))

        
    else:
        print ('Error finding solution')
        
    return(predicted,cost,points)

In [19]:
# playerOptimizer(df_hitter,df_pitcher)

In [20]:
def hitterValueOptimizer(df, hitterName, pointsH):
    # initialize variables
    df2=df.copy()
    df2.loc[df2['PlayerName'] == hitterName, 'Dollars'] = 0
    playerPositions = df2['POS'].tolist()
    playerNames = df2['PlayerName'].tolist()
    playerCosts = tuple(df2['Dollars'])
    playerPoints = tuple(df2['PTS'])
    targetPrice = 0
    
    salaryCap = 260*.7-10
    numOfPlayers = range(len(playerCosts))
    
    # formulate linear programming problemB
    problemB = pulp.LpProblem("Optimal Player Value", pulp.LpMaximize)
    # create variable to represent each player
    playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")
    targetCost=pulp.LpVariable("TC")
        
    # create objective: maximize player 'adjusted' while including him in optimal lineup
    problemB+=targetCost
    
    #constraint: our player needs to be in the lineup
    target = tuple(df2['PlayerName']==hitterName)
    problemB += pulp.lpSum(playerInLineup[i] * target[i] for i in numOfPlayers) == 1
    
    #constraint: points needs to be >= last value
    problemB += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers) >= pointsH
    
    #constraint: each player can only be chosen at most once
    for i in numOfPlayers:
        problemB += pulp.lpSum(playerInLineup[i]) <= 1
        
    # constraints: sum of player costs must be less than or equal to the salary cap
    problemB += pulp.lpSum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers)+targetCost <= salaryCap#-targetCost
    
    # constraint: teams must have 8 players
    problemB += pulp.lpSum(playerInLineup[i] for i in numOfPlayers) == 6 # num of players in lineup
    
    
    #add players from my team    
    keep = tuple(df2['Cost']>0)
    problemB += pulp.lpSum(playerInLineup[i] * keep[i] for i in numOfPlayers) == sum(keep) #num of keepers

    # positions
    # constraint: 1 = C
    #catchers = tuple(df2['Pos'] == 'C')
    catchers = tuple(df2['PP'].str.contains('C'))
    problemB += pulp.lpSum(playerInLineup[i] * catchers[i] for i in numOfPlayers) >= 0

    # constraint: 1 <= number of 1b
    fb = tuple(df2['PP'].str.contains('1B'))
    problemB += pulp.lpSum(playerInLineup[i] * fb[i] for i in numOfPlayers) >= 0

    # constraint: 1 <= number of 2b
    sb = tuple(df2['PP'].str.contains('2B'))
    problemB += pulp.lpSum(sb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of 3b
    tb = tuple(df2['PP'].str.contains('3B'))
    problemB += pulp.lpSum(tb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of SS
    ss = tuple(df2['PP'].str.contains('SS'))
    problemB += pulp.lpSum(playerInLineup[i] * ss[i] for i in numOfPlayers) >= 1
    
    # constraint: 4 <= number of OF
    of = tuple(df2['PP'].str.contains('OF'))
    problemB += pulp.lpSum(playerInLineup[i] * of[i] for i in numOfPlayers) >= 2
    
    #lazy dh fix
    dh = tuple(df2['PP']==('DH'))
    problemB += pulp.lpSum(playerInLineup[i] * dh[i] for i in numOfPlayers) == 0  
     
    # output the calculated predicted lineup
    cost = 0
    points = 0
    
    predicted = list()
    if problemB.solve() == 1:
      print ("Hypothetical Lineup")
      print (hitterName,targetCost.value())
      for pos in range(len(numOfPlayers)):
        if playerInLineup[pos].value() == 1:
          name = playerNames[pos]
          predicted.append(name)
          cost += playerCosts[pos]
          points += playerPoints[pos]      

          print ('%25s, Position = %2s, Price = %5.f, Points = %3.2f' \
                      % (playerNames[pos], playerPositions[pos], playerCosts[pos],
                         playerPoints[pos]))
        
    else:
        print ('Error finding solution')
        
    return (predicted,cost,points,targetCost.value())



In [21]:
#reset after draft
def resetDraft():
    myTeam=pd.DataFrame()
    df_hitter=df_hitter_in
    df_hitter=df_hitter.dropna(subset=['ADP']).copy()
    df_hitter['PP']=df_hitter['POS']
    positionOrder=['C','SS','2B','3B','OF','1B','SP','DH']
    df_hitter=df_hitter.copy()
    for pos in positionOrder:
        df_hitter.loc[df_hitter['PP'].str.contains(pos), 'PP'] = pos


    myRotation=pd.DataFrame()
    df_pitcher=df_pitcher_in
    df_pitcher=df_pitcher.dropna(subset=['ADP']).copy()
    df_pitcher['PP']=df_pitcher['POS']
    positionOrder2=['SP','RP','DH']
    df_pitcher=df_pitcher.copy()
    for pos in positionOrder2:
        df_pitcher.loc[df_pitcher['PP'].str.contains(pos), 'PP'] = pos

    hitter=True


In [22]:
#simulate live draft with integer number of picks
def liveDraft(picks): 
    resetDraft()
    myTeam = pd.DataFrame()
    myRotation = pd.DataFrame()
    hitter = True
    for i in range(0, picks):
        availHitter = df_hitter[df_hitter['Cost'].isnull()]
        availPitcher = df_pitcher[df_pitcher['Cost'].isnull()]

        #adjust minimum price to 1
        optiHitter = availHitter.append(myTeam).copy()
        optiHitter.loc[optiHitter['Dollars'] < 1, 'Dollars'] = 1

        #all players who have a cost have their dollars = their cost
        optiHitter.loc[optiHitter['Cost'] > 0, 'Dollars'] = optiHitter['Cost']

        # #adjust minimum price to 1
        # optiPitcher = availPitcher.append(myRotation).copy()
        # optiPitcher.loc[optiPitcher['Dollars'] < 1, 'Dollars'] = 1

        # #all players who have a cost have their adjusted = their cost
        # optiPitcher.loc[optiPitcher['Cost'] > 0, 'Dollars'] = optiPitcher['Cost']

        #lineup optimization
        # if(i==0):
        #     predictedLineup, costH, pointsH = hittingOptimizer(optiHitter)
        #     # predictedRotation, costP, pointsP = pitchingOptimizer(optiPitcher)

        if(hitter):
            predictedLineup, costH, pointsH = hittingOptimizer(optiHitter)
        # else:
        #     predictedRotation, costP, pointsP = pitchingOptimizer(optiPitcher)


        #input nominated player
        nomPlayerName = input("Please enter nominated player name: ")
        #check if player is hitter or pitcher
        # isPitcher = input("Is this player a pitcher? y/n")
        isPitcher = 'n'
        if(isPitcher=='y'):
                hitter=False
        if(hitter):
            #determine optimal cost
            predictedLineup2, costH2, pointsH2, targetCost = hitterValueOptimizer(optiHitter, nomPlayerName, pointsH)
            print(getHitter(nomPlayerName))
            # print('Best Lineup with this Player')
            # print(predictedLineup2)

            # update player cost
            auctionSold = int(input("Enter sold price: "))
            addPlayerCost(nomPlayerName,auctionSold)

            # did you draft this player
            youDrafted = input("Did you draft this player? y or n")
            if(youDrafted=='y'):
                myTeam=addHitter([nomPlayerName], myTeam)
        # else:
        #     predictedRotation2, costP2, pointsP2,targetCost = pitcherValueOptimizer(optiPitcher,nomPlayerName,pointsP)
        #     print(getPitcher(nomPlayerName))

        #     #update player cost
        #     auctionSold = int(input("Enter sold price: "))
        #     addPitcherCost(nomPlayerName,auctionSold)

        #     #did you draft this player
        #     youDrafted = input("Did you draft this player? y or n")
        #     if(youDrafted=='y'):
        #         myRotation=addPitcher([nomPlayerName], myRotation)

In [23]:
liveDraft(300)


AttributeError: 'DataFrame' object has no attribute 'to_array'

In [ ]:
mAVG = tuple(df_hitter['mAVG'])
mRBI = tuple(df_hitter['mRBI'])
mR = tuple(df_hitter['mR'])
mHR = tuple(df_hitter['mHR'])
mOPS = tuple(df_hitter['mOPS'])
mSB = tuple(df_hitter['mSBCS'])

In [ ]:


scipy.stats.cauchy.cdf(mAVG[19])

In [ ]:
scipy.stats.norm.cdf(mAVG[1]/6) 